In [1]:
import sqlite3

# Connect to SQLite database
database_path="Datasets/database/Chinook_Sqlite.sqlite"
conn = sqlite3.connect(database_path)

# Create a cursor object
cursor = conn.cursor()

```mermaid
erDiagram
  collections {
    INTEGER id PK
    TEXT title
    TEXT accession_number
    NUMERIC acquired
  }
  artists {
    INTEGER id PK
    TEXT name
  }
  created {
    INTEGER artist_id PK
    INTEGER collection_id PK
  }
  created ||--o{ collections : "collection_id → id"
  created ||--|| artists : "artist_id → id"
```

In [3]:
# Start Mermaid diagram
mermaid_output = ["erDiagram"]

# Get list of tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [table[0] for table in cursor.fetchall()]

for table in tables:
    # Get column details
    cursor.execute(f"PRAGMA table_info({table});")
    columns = cursor.fetchall()  # (cid, name, type, notnull, dflt_value, pk)
    
    # Define table in Mermaid format
    mermaid_output.append(f"  {table} {{")
    column_dict = {}  # Store column details for FK check
    for col in columns:
        col_name = col[1]
        col_type = col[2].upper()
        pk_flag = " PK" if col[5] else ""
        mermaid_output.append(f"    {col_type} {col_name}{pk_flag}")
        column_dict[col_name] = col[5]  # Track if column is a PK
    
    mermaid_output.append("  }")
    
    # Get foreign key relationships
    cursor.execute(f"PRAGMA foreign_key_list({table});")
    foreign_keys = cursor.fetchall()
    
    for fk in foreign_keys:
        from_column = fk[3]  # FK column in current table
        to_table = fk[2]      # Referenced table
        to_column = fk[4]     # Referenced column
        
        # Check if the FK column is also a primary key (1:1 relationship)
        if column_dict.get(from_column, 0) == 1:
            mermaid_output.append(f"  {table} ||--|| {to_table} : \"{from_column} → {to_column}\"")  # 1:1
        else:
            mermaid_output.append(f"  {table} ||--o{{ {to_table} : \"{from_column} → {to_column}\"")  # 1:M

# Print Mermaid syntax
mermaid_code = "\n".join(mermaid_output)
print(mermaid_code)

# Close connection
conn.close()


ProgrammingError: Cannot operate on a closed database.

```mermaid
erDiagram
  Album {
    INTEGER AlbumId PK
    NVARCHAR(160) Title
    INTEGER ArtistId
  }
  Album ||--o{ Artist : "ArtistId → ArtistId"

  Artist {
    INTEGER ArtistId PK
    NVARCHAR(120) Name
  }

  Customer {
    INTEGER CustomerId PK
    NVARCHAR(40) FirstName
    NVARCHAR(20) LastName
    NVARCHAR(80) Company
    NVARCHAR(70) Address
    NVARCHAR(40) City
    NVARCHAR(40) State
    NVARCHAR(40) Country
    NVARCHAR(10) PostalCode
    NVARCHAR(24) Phone
    NVARCHAR(24) Fax
    NVARCHAR(60) Email
    INTEGER SupportRepId
  }
  Customer ||--o{ Employee : "SupportRepId → EmployeeId"

  Employee {
    INTEGER EmployeeId PK
    NVARCHAR(20) LastName
    NVARCHAR(20) FirstName
    NVARCHAR(30) Title
    INTEGER ReportsTo
    DATETIME BirthDate
    DATETIME HireDate
    NVARCHAR(70) Address
    NVARCHAR(40) City
    NVARCHAR(40) State
    NVARCHAR(40) Country
    NVARCHAR(10) PostalCode
    NVARCHAR(24) Phone
    NVARCHAR(24) Fax
    NVARCHAR(60) Email
  }
  Employee ||--o{ Employee : "ReportsTo → EmployeeId"

  Genre {
    INTEGER GenreId PK
    NVARCHAR(120) Name
  }

  Invoice {
    INTEGER InvoiceId PK
    INTEGER CustomerId
    DATETIME InvoiceDate
    NVARCHAR(70) BillingAddress
    NVARCHAR(40) BillingCity
    NVARCHAR(40) BillingState
    NVARCHAR(40) BillingCountry
    NVARCHAR(10) BillingPostalCode
    FLOAT Total
  }
  Invoice ||--o{ Customer : "CustomerId → CustomerId"

  InvoiceLine {
    INTEGER InvoiceLineId PK
    INTEGER InvoiceId
    INTEGER TrackId
    FLOAT UnitPrice
    INTEGER Quantity
  }
  InvoiceLine ||--o{ Track : "TrackId → TrackId"
  InvoiceLine ||--o{ Invoice : "InvoiceId → InvoiceId"

  MediaType {
    INTEGER MediaTypeId PK
    NVARCHAR(120) Name
  }

  Playlist {
    INTEGER PlaylistId PK
    NVARCHAR(120) Name
  }

  PlaylistTrack {
    INTEGER PlaylistId PK
    INTEGER TrackId PK
  }
  PlaylistTrack ||--o{ Track : "TrackId → TrackId"
  PlaylistTrack ||--|| Playlist : "PlaylistId → PlaylistId"

  Track {
    INTEGER TrackId PK
    NVARCHAR(200) Name
    INTEGER AlbumId
    INTEGER MediaTypeId
    INTEGER GenreId
    NVARCHAR(220) Composer
    INTEGER Milliseconds
    INTEGER Bytes
    FLOAT UnitPrice
  }
  Track ||--o{ MediaType : "MediaTypeId → MediaTypeId"
  Track ||--o{ Genre : "GenreId → GenreId"
  Track ||--o{ Album : "AlbumId → AlbumId"
```